In [ ]:
import time
import copy
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import grad
from torch.autograd import Variable

In [ ]:
torch.set_default_dtype(torch.float)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Sin(nn.Module):
  """
  Hamiltonian neural networks for solving differential equations https://arxiv.org/pdf/2001.11107
  The use of sin(·) instead of more common activation functions, such as Sigmoid(·) and tanh(·), 
  significantly accelerates the network's convergence to a solution
  """
  def __init__(self):
    pass
  def forward(self, x):
    return torch.sin(x)

In [ ]:
def compute_gradient(f, x):
    """
    One-dimensional stationary Schrödinger's equation has the second order derivative
    So we have to compute d/dx (d/dx f)
    This has to be done explicitly and will not be covered by the loss.backward() which will 
    actually compute the gradient of the loss with regards to the model parameters theta
    """
    grad_outputs = torch.ones_like(f, device=x.device)
    gradient = torch.autograd.grad(outputs=f, inputs=x, grad_outputs=grad_outputs, create_graph=True)[0]
    return gradient

In [ ]:
def perturb_grid_points(grid, xL, xR, perturbation_factor):
    """
    For the training, a batch of xi points in the interval [xL, xR] is selected as input. In every training iteration (epoch) the
    input points are perturbed by a Gaussian noise to prevent the network from learning the solutions only at fixed points.
    """
    delta = grid[1] - grid[2]

    noise = delta * torch.randn_like(grid) * perturbation_factor
    perturbed_grid = grid + noise

    perturbed_grid[perturbed_grid < xL] = 2*xL - perturbed_grid[perturbed_grid < xL]
    perturbed_grid[perturbed_grid > xR] = 2*xR - perturbed_grid[perturbed_grid > xR]

    perturbed_grid[0] = xL
    perturbed_grid[-1] = xR

    perturbed_grid.requires_grad = False

    return perturbed_grid

In [ ]:
def parametric_function(x, xL, xR):
    """
    Selecting an appropriate parametric function g(x) is necessary for enforcing boundary conditions. 
    The following parametric equation enforces a f (xL) = f (xR) = 0 boundary conditions
    """
    return (1 - torch.exp(-(x - xL))) * (1 - torch.exp(-(x - xR)))


def parametric_trick(x, xL, xR, boundary_value, neural_net):
    """
    The predicted eigenfunctions are defined using a parametric trick, i.e are not directly computed by the neural network
    The neural network returns a tuple: network_prediction and learnable_lambda
    """
    network_prediction, _ = neural_net(x)
    g = parametric_function(x, xL, xR)
    return boundary_value + g * network_prediction


In [ ]:
def compute_schroedinger_loss_residual_and_hamiltonian(x, psi, E, V):
    """
    Schrödingers loss is the mean of the residual. Hamiltonian is used for verification. 
    The main reason to couple the computations is to use only one autograd pass
    """
    psi_dx = compute_gradient(psi, x)
    psi_ddx = compute_gradient(psi_dx, x)

    residual = -0.5 * psi_ddx + (V - E) * psi
    schroeddinger_loss = (residual.pow(2)).mean()
    hamiltonian = -0.5 * psi_ddx + V * psi

    return schroeddinger_loss, residual, hamiltonian


In [ ]:
class Quantum_NN(nn.Module):
    """
    Network for the 1D PINN task
    """
    def __init__(self, hidden_dim=10, symmetry=True):
        super().__init__()
        self.symmetry = symmetry

        self.activation = Sin()
        self.input = nn.Linear(1, 1)
        self.first_hidden = nn.Linear(2, hidden_dim)
        self.second_hidden = nn.Linear(hidden_dim + 1, hidden_dim)
        self.output = nn.Linear(hidden_dim + 1, 1)

    def forward(self, t):
        learnable_lambda = self.input(torch.ones_like(t))

        first_hidden_symmetric = self.activation(self.first_hidden(torch.cat((t, learnable_lambda), dim=1)))
        first_hidden_antisymmetric = self.activation(self.first_hiden(torch.cat((-t, learnable_lambda), dim=1)))

        secondd_hidden_symmetric = self.activation(self.second_hidden(torch.cat((first_hidden_symmetric, learnable_lambda), dim=1)))
        second_hidden_antisymmetric = self.activation(self.second_hidden(torch.cat((first_hidden_antisymmetric, learnable_lambda), dim=1)))

        if self.symmetry:
            network_prediction = self.output(torch.cat((secondd_hidden_symmetric + second_hidden_antisymmetric, learnable_lambda), dim=1))
        else:
            network_prediction = self.output(torch.cat((secondd_hidden_symmetric - second_hidden_antisymmetric, learnable_lambda), dim=1))

        return network_prediction, learnable_lambda